In [1]:
import pandas as pd

### Read csv, make copy of csv, change panda display options to see the entire dataset

In [2]:
raw_data = pd.read_csv('Exoplanet_Composite_Raw_Data.csv')
filtered_raw_data = raw_data
pd.options.display.max_columns=None
pd.options.display.max_rows=None
filtered_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5602 entries, 0 to 5601
Columns: 193 entries, pl_name to pl_ntranspec
dtypes: float64(166), int64(21), object(6)
memory usage: 8.2+ MB


## Data Preprocessing - 
### Set threshold to 7% : if 7% or more of feature is missing values, we exclude the column 

In [3]:
filtered_raw_data.shape

(5602, 193)

### Shape tells us there are 5602 rows and 84 columns

In [4]:
threshold = round(filtered_raw_data.shape[0] * .07)
threshold

392

### Get an idea of how many missing values are in each column currently

In [5]:
filtered_raw_data.isnull().sum()

pl_name               0
hostname              0
sy_snum               0
sy_pnum               0
sy_mnum               0
cb_flag               0
discoverymethod       0
disc_year             0
disc_facility         0
disc_telescope        0
disc_instrument       0
rv_flag               0
pul_flag              0
ptv_flag              0
tran_flag             0
ast_flag              0
obm_flag              0
micro_flag            0
etv_flag              0
ima_flag              0
dkin_flag             0
pl_controv_flag       0
pl_orbper           249
pl_orbpererr1       728
pl_orbpererr2       728
pl_orbperlim        249
pl_rade              20
pl_radeerr1        1754
pl_radeerr2        1754
pl_radelim           20
pl_bmasse            26
pl_bmasseerr1      3094
pl_bmasseerr2      3094
pl_bmasselim         26
pl_dens             110
pl_denserr1        4589
pl_denserr2        4589
pl_denslim          110
pl_orbeccen         777
pl_orbeccenerr1    3986
pl_orbeccenerr2    3986
pl_orbeccenlim  

### Drop columns that meet or exceed the threshold

In [6]:
for col in filtered_raw_data:
    if filtered_raw_data[col].isnull().sum() >= threshold:
        filtered_raw_data = filtered_raw_data.drop([col], axis=1)

In [7]:
filtered_raw_data.isnull().sum()

pl_name              0
hostname             0
sy_snum              0
sy_pnum              0
sy_mnum              0
cb_flag              0
discoverymethod      0
disc_year            0
disc_facility        0
disc_telescope       0
disc_instrument      0
rv_flag              0
pul_flag             0
ptv_flag             0
tran_flag            0
ast_flag             0
obm_flag             0
micro_flag           0
etv_flag             0
ima_flag             0
dkin_flag            0
pl_controv_flag      0
pl_orbper          249
pl_orbperlim       249
pl_rade             20
pl_radelim          20
pl_bmasse           26
pl_bmasselim        26
pl_dens            110
pl_denslim         110
st_teff            219
st_tefflim         219
st_rad             240
st_radlim          240
st_mass              4
st_masserr1        270
st_masserr2        291
st_masslim           4
st_lum             235
st_lumlim          235
st_logg            246
st_logglim         246
glat                 0
glon       

### Read master column mapper csv, assort keys and values in a dictionary and update filtered raw data columns accordingly

In [8]:
column_mapper = pd.read_csv('Exoplanet_Archive_Column_Mapping.csv')
mapper = {'pl_name' : 'Planet Name'} 
# pl_name.1 and Planet Name.1 are the relevant columns for the composite dataset
db_col_name, table_label = column_mapper[' pl_name.1'], column_mapper['Planet Name.1']

### Loop through number of code columns (ex: pl_name) and their definitions (ex: planet name)

In [9]:
for i in range(358):
    # if the row i in the database column name matches a column name in our filtered raw data, we assign the key and value in our dict 
    if str(db_col_name[i]).strip() in filtered_raw_data.columns.values:
        mapper[db_col_name[i].strip()] = table_label[i].strip()
        # use strip to remove trailing and leading whitespaces

### Double check we mapped all database column names to their definitions 

In [10]:
len(mapper) == len(filtered_raw_data.columns.values)

True

### Map Columns

In [11]:
filtered_raw_data.columns= mapper.values()

### Checkpoint after removing invalid columns and renaming the remaining columns

In [12]:
filtered_renamed_data = filtered_raw_data
filtered_renamed_data.head()

,Planet Name,Host Name,Number of Stars,Number of Planets,Number of Moons,Circumbinary Flag,Discovery Method,Discovery Year,Discovery Facility,Discovery Telescope,Discovery Instrument,Detected by Radial Velocity Variations,Detected by Pulsar Timing Variations,Detected by Pulsation Timing Variations,Detected by Transits,Detected by Astrometric Variations,Detected by Orbital Brightness Modulations,Detected by Microlensing,Detected by Eclipse Timing Variations,Detected by Imaging,Detected by Disk Kinematics,Controversial Flag,Orbital Period [days],Orbital Period Limit Flag,Planet Radius [Earth Radius],Planet Radius Limit Flag,Planet Mass or Mass*sin(i) [Earth Mass],Planet Mass or Mass*sin(i) [Earth Mass] Limit Flag,Planet Density [g/cm**3],Planet Density Limit Flag,Stellar Effective Temperature [K],Stellar Effective Temperature Limit Flag,Stellar Radius [Solar Radius],Stellar Radius Limit Flag,Stellar Mass [Solar mass],Stellar Mass Upper Unc. [Solar mass],Stellar Mass Lower Unc. [Solar mass],Stellar Mass Limit Flag,Stellar Luminosity [log(Solar)],Stellar Luminosity Limit Flag,Stellar Surface Gravity [log10(cm/s**2)],Stellar Surface Gravity Limit Flag,Galactic Latitude [deg],Galactic Longitude [deg],Ecliptic Latitude [deg],Ecliptic Longitude [deg],Total Proper Motion [mas/yr],Total Proper Motion Upper Unc [mas/yr],Total Proper Motion Lower Unc [mas/yr],Proper Motion (RA) [mas/yr],Proper Motion (RA) [mas/yr] Upper Unc,Proper Motion (RA) [mas/yr] Lower Unc,Proper Motion (Dec) [mas/yr],Proper Motion (Dec) [mas/yr] Upper Unc,Proper Motion (Dec) [mas/yr] Lower Unc,Distance [pc],Distance [pc] Upper Unc,Distance [pc] Lower Unc,B (Johnson) Magnitude,B (Johnson) Magnitude Upper Unc,B (Johnson) Magnitude Lower Unc,V (Johnson) Magnitude,V (Johnson) Magnitude Upper Unc,V (Johnson) Magnitude Lower Unc,J (2MASS) Magnitude,J (2MASS) Magnitude Upper Unc,J (2MASS) Magnitude Lower Unc,H (2MASS) Magnitude,H (2MASS) Magnitude Upper Unc,H (2MASS) Magnitude Lower Unc,Ks (2MASS) Magnitude,Ks (2MASS) Magnitude Upper Unc,Ks (2MASS) Magnitude Lower Unc,Gaia Magnitude,Gaia Magnitude Upper Unc,Gaia Magnitude Lower Unc,TESS Magnitude,TESS Magnitude Upper Unc,TESS Magnitude Lower Unc,Number of Photometry Time Series,Number of Radial Velocity Time Series,Number of Stellar Spectra Measurements,Number of Emission Spectroscopy Measurements,Number of Transmission Spectroscopy Measurements
0,11 Com b,11 Com,2,1,0,0,Radial Velocity,2007,Xinglong Station,2.16 m Telescope,Coude Echelle Spectrograph,1,0,0,0,0,0,0,0,0,0,0,323.21000,0.0,12.2,0.0,4914.89849,0.0,14.90,0.0,4874.0,0.0,13.76,0.0,2.09,0.64,-0.63,0.0,1.978,0.0,2.45,0.0,78.28058,264.13775,18.33392,177.41790,140.383627,0.301395,-0.301395,-109.24100,0.316107,-0.316107,88.1701,0.277297,-0.277297,93.1846,1.9238,-1.9238,5.726,0.017000,-0.017000,4.72307,0.023,-0.023,2.943,0.334,-0.334,2.484,0.268,-0.268,2.282,0.346,-0.346,4.44038,0.003848,-0.003848,3.83790,0.0071,-0.0071,1,2,0,0,0
1,11 UMi b,11 UMi,1,1,0,0,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,Coude Echelle Spectrograph,1,0,0,0,0,0,0,0,0,0,0,516.21997,0.0,12.3,0.0,4684.81420,0.0,13.80,0.0,4213.0,0.0,29.79,0.0,2.78,0.69,-0.69,0.0,2.430,0.0,1.93,0.0,41.04437,108.71900,74.95821,141.64699,10.679019,0.218806,-0.218806,3.42996,0.231822,-0.231822,10.1132,0.217259,-0.217259,125.3210,1.9765,-1.9765,6.382,0.045000,-0.045000,5.01300,0.005,-0.005,2.876,0.230,-0.230,2.091,0.194,-0.194,1.939,0.270,-0.270,4.56216,0.003903,-0.003903,3.82294,0.0075,-0.0075,1,1,0,0,0
2,14 And b,14 And,1,1,0,0,Radial Velocity,2008,Okayama Astrophysical Observatory,1.88 m Telescope,HIDES Echelle Spectrograph,1,0,0,0,0,0,0,0,0,0,0,186.76000,0.0,13.1,0.0,1131.15130,0.0,2.76,0.0,4888.0,0.0,11.55,0.0,1.78,0.43,-0.29,0.0,1.840,0.0,2.55,0.0,-21.05141,106.41269,38.22901,11.95935,299.252618,0.182684,-0.182684,287.29200,0.185780,-0.185780,-83.7582,0.141256,-0.141256,75.4392,0.7140,-0.7140,6.256,0.017000,-0.017000,5.23133,0.023,-0.023,3.019,0.180,-0.180,2.608,0.184

### Get an idea of the unique values we can expect in the discovery method column ; 11 unique values

In [13]:
filtered_renamed_data['Discovery Method'].unique()

array(['Radial Velocity', 'Imaging', 'Eclipse Timing Variations',
       'Transit', 'Transit Timing Variations', 'Astrometry',
       'Disk Kinematics', 'Microlensing', 'Orbital Brightness Modulation',
       'Pulsation Timing Variations', 'Pulsar Timing'], dtype=object)

### The dataset is missing transit timing variations as a target dummy column ; This is expected to prevent perfect multicollinearity
### (We always want n-1 dummies for n categories so the omitted category serves as the reference category against which the others are compared)
### We keep the all dummy target columns for model training and drop Discovery Method column to prevent multicollinearity

### Get an idea of the frequency of each value in the discovery method column

In [14]:
filtered_renamed_data['Discovery Method'].value_counts()

Discovery Method
Transit                          4168
Radial Velocity                  1088
Microlensing                      210
Imaging                            68
Transit Timing Variations          29
Eclipse Timing Variations          17
Orbital Brightness Modulation       9
Pulsar Timing                       7
Astrometry                          3
Pulsation Timing Variations         2
Disk Kinematics                     1
Name: count, dtype: int64

### Clear discrepancy ; transit has 4168 occurrences, meanwhile Orbital Brightness Modulation, Pulsar Timing, Astrometry,
### Pulsation Timming Variations and Disk Kinematics all have less than 10 occurrences 
### We would usually need to balance the data, doing so using SMOTE (Synthetic Minority Over-Sampling Technique) 
### This would synthesize instances of minority classes so that training will consider all classes evenly

In [15]:
print(len(filtered_renamed_data['Discovery Facility'].value_counts()),
       len(filtered_renamed_data['Discovery Telescope'].value_counts()), 
          len(filtered_renamed_data['Discovery Instrument'].value_counts()))

70 79 94


### We could encode Discovery Facility, Discovery Telescope,, Discovery Instrument, but they each have 70-90 unique values

### To reduce dimensionality of the feature space and prevent multicollinearity we will exclude them for now
### Thus we will drop Planet Name, Host Name, Discovery Facility, Discovery Telescope and Discovery Instrument 
### We will later confirm their impact on the model through Cross-Validation and Feature Importance Analysis

In [16]:
unnecessary_features = ['Planet Name',
                        'Host Name',
                        'Discovery Method',
                        'Discovery Facility',
                        'Discovery Telescope',
                        'Discovery Instrument']

preprocessed_data = filtered_renamed_data.drop(unnecessary_features, axis=1)


In [17]:
preprocessed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5602 entries, 0 to 5601
Data columns (total 78 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Number of Stars                                     5602 non-null   int64  
 1   Number of Planets                                   5602 non-null   int64  
 2   Number of Moons                                     5602 non-null   int64  
 3   Circumbinary Flag                                   5602 non-null   int64  
 4   Discovery Year                                      5602 non-null   int64  
 5   Detected by Radial Velocity Variations              5602 non-null   int64  
 6   Detected by Pulsar Timing Variations                5602 non-null   int64  
 7   Detected by Pulsation Timing Variations             5602 non-null   int64  
 8   Detected by Transits                                5602 non-null   int64  
 9

In [18]:
# finally export the new csv 
exoplanet_composite_preprocessed = preprocessed_data
exoplanet_composite_preprocessed.to_csv('Composite_preprocessed.csv', index=False)